In [1]:
import json
import logging
import math
import random
import time
from collections import defaultdict
from contextlib import contextmanager
from datetime import datetime
from pathlib import Path
from typing import Any, Dict, List, Optional, Tuple

import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import pandas as pd
import seaborn as sns
from tqdm import tqdm

In [2]:
# Configure logging
logger = logging.getLogger(__name__)

## Functions

In [3]:
class MetricStatus:
    """Enumeration for metric computation status."""
    COMPUTED = "✅ computed"
    APPROXIMATED = "⚠️ approximated"
    FAILED = "❌ failed"
    SKIPPED = "⏭️ skipped"


class MetricTimer:
    """Context manager for timing metric computations."""

    def __init__(self, metric_name: str, logger: logging.Logger):
        self.metric_name = metric_name
        self.logger = logger
        self.start_time = None
        self.end_time = None

    def __enter__(self):
        self.start_time = time.time()
        self.logger.info(f"Computing {self.metric_name}...")
        return self

    def __exit__(self, exc_type, exc_val, exc_tb):
        self.end_time = time.time()
        duration = self.end_time - self.start_time
        if exc_type is None:
            self.logger.info(
                f"✅ {self.metric_name} completed in {(duration)}")
        else:
            self.logger.warning(
                f"❌ {self.metric_name} failed after {(duration)}: {exc_val}")

    @property
    def duration(self):
        if self.end_time and self.start_time:
            return self.end_time - self.start_time
        return None


class FastDistanceAlgorithms:
    """Fast approximation algorithms for distance metrics."""

    @staticmethod
    def approximate_diameter_2sweep(G: nx.Graph) -> Tuple[int, str]:
        """
        Fast 2-sweep approximation for diameter.
        Time: O(n + m) vs O(n^2) exact
        """
        if G.number_of_nodes() <= 1:
            return 0, "exact"

        # First BFS from random node
        start_node = random.choice(list(G.nodes()))
        distances = nx.single_source_shortest_path_length(G, start_node)

        # Find furthest node
        furthest_node = max(distances.items(), key=lambda x: x[1])[0]

        # Second BFS from furthest node
        distances2 = nx.single_source_shortest_path_length(G, furthest_node)
        diameter_approx = max(distances2.values())

        return diameter_approx, "2-sweep approximation"

    @staticmethod
    def approximate_avg_path_sampling(G: nx.Graph, sample_size: int = 1000) -> Tuple[float, str]:
        """Sample-based average shortest path length estimation."""
        nodes = list(G.nodes())
        n = len(nodes)

        if n <= 1:
            return 0.0, "exact"

        # Adaptive sample size
        actual_sample = min(sample_size, max(100, n * n // 1000))
        path_lengths = []

        for _ in range(actual_sample):
            source, target = random.sample(nodes, 2)
            try:
                path_length = nx.shortest_path_length(G, source, target)
                path_lengths.append(path_length)
            except nx.NetworkXNoPath:
                continue

        if not path_lengths:
            return float('inf'), "disconnected"

        return np.mean(path_lengths), f"sampling ({len(path_lengths)} pairs)"

    @staticmethod
    def approximate_radius_sampling(G: nx.Graph, sample_size: int = 100) -> Tuple[int, str]:
        """Fast radius approximation using node sampling."""
        nodes = list(G.nodes())
        n = len(nodes)

        if n <= 1:
            return 0, "exact"

        # Sample potential center nodes
        sample_nodes = random.sample(nodes, min(sample_size, n))
        min_eccentricity = float('inf')

        for center in sample_nodes:
            try:
                distances = nx.single_source_shortest_path_length(G, center)
                eccentricity = max(distances.values())
                min_eccentricity = min(min_eccentricity, eccentricity)
            except:
                continue

        if min_eccentricity == float('inf'):
            return float('inf'), "disconnected"

        return int(min_eccentricity), f"sampling ({len(sample_nodes)} centers)"

    @staticmethod
    def estimate_global_efficiency(G: nx.Graph, sample_size: int = 1000) -> Tuple[float, str]:
        """Fast global efficiency estimation via sampling."""
        nodes = list(G.nodes())
        n = len(nodes)

        if n <= 1:
            return 0.0, "exact"

        efficiency_values = []
        actual_sample = min(sample_size, max(100, n * n // 1000))

        for _ in range(actual_sample):
            source, target = random.sample(nodes, 2)
            try:
                path_length = nx.shortest_path_length(G, source, target)
                if path_length > 0:
                    efficiency_values.append(1.0 / path_length)
            except nx.NetworkXNoPath:
                efficiency_values.append(0.0)

        if not efficiency_values:
            return 0.0, "no paths"

        return np.mean(efficiency_values), f"sampling ({len(efficiency_values)} pairs)"

## Graph

In [14]:
import ast


def create_test_graph():
    """Create test graph with metadata."""
    # Load dataset
    df = pd.read_csv("../data/v0.0/df_nq_version0.csv")

    # Convert stringified lists into real Python lists
    df["cites_ids"] = df["cites_ids"].apply(ast.literal_eval)

    # Create a directed graph
    G = nx.DiGraph()

    # Add edges to the graph
    for _, row in df.iterrows():
        src = row["id"]
        G.add_node(src, title=row["d_properties_document_title"])
        for tgt in row["cites_ids"]:
            G.add_edge(src, tgt)

    # Graph metadata for reporting
    graph_info = {
        "source": "NQ (Natural Questions) from Google",
        "description": "Graph constructed from wiki articles citation data in the Natural Questions dataset"
    }

    return G, graph_info

G, _ = create_test_graph()

In [23]:
percentage = 0.25
nodes = list(G.nodes())
n = len(nodes)
sample_size = max(1, int(n * percentage))

nodes_list = list(G.nodes())
sample_nodes = random.sample(nodes_list, min(sample_size, len(nodes_list)))
sample_graph = G.subgraph(sample_nodes).copy()

print(f"Graph Information:")
print(f"- Nodes: {sample_graph.number_of_nodes():,}")
print(f"- Edges: {sample_graph.number_of_edges():,}")
print(f"- Directed: {sample_graph.is_directed()}")

Graph Information:
- Nodes: 27,017
- Edges: 335,276
- Directed: True


In [32]:

sccs = list(nx.strongly_connected_components(G))
largest_scc = max(sccs, key=len)
analysis_graph = G.subgraph(largest_scc).copy()
print(f"Graph Information:")
print(f"- Nodes: {analysis_graph.number_of_nodes():,}")
print(f"- Edges: {analysis_graph.number_of_edges():,}")
print(f"- Directed: {analysis_graph.is_directed()}")

def bfs_sample(G, start, sample_size):
    visited = {start}
    queue = [start]
    while queue and len(visited) < sample_size:
        v = queue.pop(0)
        for u in G.neighbors(v):
            if u not in visited:
                visited.add(u)
                queue.append(u)
                if len(visited) >= sample_size:
                    break
    return G.subgraph(visited).copy()

percentage = 0.25
start = random.choice(list(analysis_graph.nodes()))
sample_graph = bfs_sample(analysis_graph, start, int(len(analysis_graph) * percentage))

print(f"Graph Information:")
print(f"- Nodes: {sample_graph.number_of_nodes():,}")
print(f"- Edges: {sample_graph.number_of_edges():,}")
print(f"- Directed: {sample_graph.is_directed()}")
print(f"- Connected: {nx.is_connected(sample_graph.to_undirected())}")

Graph Information:
- Nodes: 94,675
- Edges: 4,900,260
- Directed: True
Graph Information:
- Nodes: 23,668
- Edges: 1,722,419
- Directed: True
- Connected: True


In [38]:
sccs = list(nx.strongly_connected_components(sample_graph))
largest_scc = max(sccs, key=len)
analysis_graph2 = sample_graph.subgraph(largest_scc).copy()
print(f"Graph Information:")
print(f"- Nodes: {analysis_graph2.number_of_nodes():,}")
print(f"- Edges: {analysis_graph2.number_of_edges():,}")
print(f"- Directed: {analysis_graph2.is_directed()}")

Graph Information:
- Nodes: 23,659
- Edges: 1,722,304
- Directed: True


In [36]:
from networkx.algorithms import approximation as approx

print(approx.diameter(analysis_graph.to_undirected()))

5


In [39]:
print(nx.average_shortest_path_length(analysis_graph2))

3.032798709005158


In [40]:
print(nx.radius(analysis_graph2))

4


In [44]:
start = random.choice(list(analysis_graph.nodes()))
sample_graph = bfs_sample(analysis_graph, start, int(len(analysis_graph) * 0.15))

print(f"Graph Information:")
print(f"- Nodes: {sample_graph.number_of_nodes():,}")
print(f"- Edges: {sample_graph.number_of_edges():,}")
print(f"- Directed: {sample_graph.is_directed()}")
print(f"- Connected: {nx.is_connected(sample_graph.to_undirected())}")

sccs = list(nx.strongly_connected_components(sample_graph))
largest_scc = max(sccs, key=len)
analysis_graph3 = sample_graph.subgraph(largest_scc).copy()
print(f"Graph Information:")
print(f"- Nodes: {analysis_graph3.number_of_nodes():,}")
print(f"- Edges: {analysis_graph3.number_of_edges():,}")
print(f"- Directed: {analysis_graph3.is_directed()}")

Graph Information:
- Nodes: 14,201
- Edges: 1,115,438
- Directed: True
- Connected: True
Graph Information:
- Nodes: 14,197
- Edges: 1,115,381
- Directed: True


In [45]:
print(nx.average_shortest_path_length(analysis_graph3))

2.818837962048066


In [46]:
print(nx.radius(analysis_graph3))


3


In [47]:
print(nx.global_efficiency(analysis_graph3.to_undirected()))

0.46832696495964377


In [48]:
start = random.choice(list(analysis_graph.nodes()))
sample_graph = bfs_sample(analysis_graph, start, int(len(analysis_graph) * 0.20))

print(f"Graph Information:")
print(f"- Nodes: {sample_graph.number_of_nodes():,}")
print(f"- Edges: {sample_graph.number_of_edges():,}")
print(f"- Directed: {sample_graph.is_directed()}")
print(f"- Connected: {nx.is_connected(sample_graph.to_undirected())}")

sccs = list(nx.strongly_connected_components(sample_graph))
largest_scc = max(sccs, key=len)
analysis_graph4 = sample_graph.subgraph(largest_scc).copy()
print(f"Graph Information:")
print(f"- Nodes: {analysis_graph4.number_of_nodes():,}")
print(f"- Edges: {analysis_graph4.number_of_edges():,}")
print(f"- Directed: {analysis_graph4.is_directed()}")

Graph Information:
- Nodes: 18,935
- Edges: 1,423,749
- Directed: True
- Connected: True
Graph Information:
- Nodes: 18,933
- Edges: 1,423,737
- Directed: True


In [49]:
print(nx.average_shortest_path_length(analysis_graph4))


2.998498633895194


In [50]:
print(nx.radius(analysis_graph4))


3


In [51]:
print(nx.global_efficiency(analysis_graph4.to_undirected()))


0.4627325813771752


In [55]:
start = random.choice(list(analysis_graph.nodes()))
sample_graph = bfs_sample(analysis_graph, start, int(len(analysis_graph) * 0.05))

print(f"Graph Information:")
print(f"- Nodes: {sample_graph.number_of_nodes():,}")
print(f"- Edges: {sample_graph.number_of_edges():,}")
print(f"- Directed: {sample_graph.is_directed()}")
print(f"- Connected: {nx.is_connected(sample_graph.to_undirected())}")

sccs = list(nx.strongly_connected_components(sample_graph))
largest_scc = max(sccs, key=len)
analysis_graph5 = sample_graph.subgraph(largest_scc).copy()
print(f"Graph Information:")
print(f"- Nodes: {analysis_graph5.number_of_nodes():,}")
print(f"- Edges: {analysis_graph5.number_of_edges():,}")
print(f"- Directed: {analysis_graph5.is_directed()}")

Graph Information:
- Nodes: 4,733
- Edges: 397,460
- Directed: True
- Connected: True
Graph Information:
- Nodes: 4,729
- Edges: 397,450
- Directed: True


In [56]:
print(nx.average_shortest_path_length(analysis_graph5))


2.5717285950997537


In [57]:
print(nx.radius(analysis_graph5))


3


In [58]:
print(nx.global_efficiency(analysis_graph5.to_undirected()))


0.49387526586820557


In [5]:
# Extract and analyze the graph
G_graph, graph_info = G
print(f"Graph Information:")
print(f"- Nodes: {G_graph.number_of_nodes():,}")
print(f"- Edges: {G_graph.number_of_edges():,}")
print(f"- Directed: {G_graph.is_directed()}")
print(f"- Source: {graph_info['source']}")
print(f"- Description: {graph_info['description']}")

# Check connectivity and components
if G_graph.is_directed():
    is_connected = nx.is_strongly_connected(G_graph)
    components = list(nx.strongly_connected_components(G_graph))
    conn_type = "strongly connected"
else:
    is_connected = nx.is_connected(G_graph)
    components = list(nx.connected_components(G_graph))
    conn_type = "connected"

print(f"\nConnectivity Analysis:")
print(f"- Is {conn_type}: {is_connected}")
print(f"- Number of components: {len(components)}")

if components:
    component_sizes = [len(comp) for comp in components]
    largest_component_size = max(component_sizes)
    print(f"- Largest component size: {largest_component_size:,} nodes")
    print(f"- Largest component ratio: {largest_component_size/G_graph.number_of_nodes():.4f}")
    
    # Store the largest component for experiments
    largest_component = max(components, key=len)
    G_main = G_graph.subgraph(largest_component).copy()
    print(f"\nWorking with largest component: {G_main.number_of_nodes():,} nodes, {G_main.number_of_edges():,} edges")
else:
    G_main = G_graph
    print("Working with full graph")

Graph Information:
- Nodes: 108,071
- Edges: 5,122,983
- Directed: True
- Source: NQ (Natural Questions) from Google
- Description: Graph constructed from wiki articles citation data in the Natural Questions dataset

Connectivity Analysis:
- Is strongly connected: False
- Number of components: 13288
- Largest component size: 94,675 nodes
- Largest component ratio: 0.8760

Working with largest component: 94,675 nodes, 4,900,260 edges


## Sample Class

In [6]:
class SampleSizeExperiment:
    """Framework for testing optimal sample sizes for distance metrics."""
    
    def __init__(self, graph: nx.Graph, max_time_threshold: float = 300.0):
        """
        Initialize experiment framework.
        
        Args:
            graph: NetworkX graph for experiments
            max_time_threshold: Maximum acceptable computation time in seconds
        """
        self.graph = graph
        self.max_time_threshold = max_time_threshold
        self.results = []
        self.fast_algo = FastDistanceAlgorithms()
        
        # Base graph properties
        self.total_nodes = graph.number_of_nodes()
        self.total_edges = graph.number_of_edges()
        
        print(f"Experiment Setup:")
        print(f"- Graph: {self.total_nodes:,} nodes, {self.total_edges:,} edges")
        print(f"- Max time threshold: {max_time_threshold:.1f} seconds")
        
    def generate_sample_sizes(self, start_samples=100, max_samples=10000, step_multiplier=1.5):
        """Generate increasing sample sizes for testing."""
        sample_sizes = []
        current = start_samples
        
        while current <= max_samples:
            sample_sizes.append(int(current))
            current *= step_multiplier
            
        # Add a few more aggressive sizes if graph is large enough
        if self.total_nodes > 50000:
            additional_sizes = [15000, 20000, 25000, 30000]
            sample_sizes.extend([s for s in additional_sizes if s <= max_samples])
            
        return sorted(set(sample_sizes))
    
    def test_metric_scalability(self, metric_name: str, metric_func, sample_sizes: List[int], 
                              num_runs: int = 3, **func_kwargs):
        """
        Test a specific metric across different sample sizes.
        
        Args:
            metric_name: Name of the metric being tested
            metric_func: Function to compute the metric
            sample_sizes: List of sample sizes to test
            num_runs: Number of runs per sample size for averaging
            **func_kwargs: Additional kwargs for the metric function
        """
        print(f"\n{'='*60}")
        print(f"Testing {metric_name.upper()} scalability")
        print(f"{'='*60}")
        
        metric_results = []
        nodes_list = list(self.graph.nodes())
        
        for sample_size in sample_sizes:
            print(f"\nTesting sample size: {sample_size:,} ({sample_size/self.total_nodes:.4f} of total)")
            
            run_times = []
            run_values = []
            run_methods = []
            
            for run in range(num_runs):
                print(f"  Run {run + 1}/{num_runs}...", end=" ")
                
                try:
                    # Create sample subgraph
                    sample_nodes = random.sample(nodes_list, min(sample_size, len(nodes_list)))
                    sample_graph = self.graph.subgraph(sample_nodes).copy()
                    
                    # Make sure sample is connected (take largest component)
                    if sample_graph.is_directed():
                        if not nx.is_strongly_connected(sample_graph):
                            components = list(nx.strongly_connected_components(sample_graph))
                            if components:
                                largest_comp = max(components, key=len)
                                sample_graph = sample_graph.subgraph(largest_comp).copy()
                    else:
                        if not nx.is_connected(sample_graph):
                            components = list(nx.connected_components(sample_graph))
                            if components:
                                largest_comp = max(components, key=len)
                                sample_graph = sample_graph.subgraph(largest_comp).copy()
                    
                    # Convert to undirected for distance calculations
                    distance_graph = sample_graph.to_undirected() if sample_graph.is_directed() else sample_graph
                    
                    # Time the computation
                    start_time = time.time()
                    result = metric_func(distance_graph, **func_kwargs)
                    end_time = time.time()
                    
                    computation_time = end_time - start_time
                    run_times.append(computation_time)
                    
                    # Handle different return types
                    if isinstance(result, tuple):
                        value, method = result
                        run_values.append(value)
                        run_methods.append(method)
                    else:
                        run_values.append(result)
                        run_methods.append("exact")
                    
                    print(f"✅ {computation_time:.2f}s")
                    
                    # Early termination if taking too long
                    if computation_time > self.max_time_threshold:
                        print(f"    ⚠️ Exceeded time threshold ({self.max_time_threshold}s), stopping here")
                        break
                        
                except Exception as e:
                    print(f"❌ Error: {str(e)[:50]}")
                    continue
            
            if run_times:
                # Calculate statistics
                avg_time = np.mean(run_times)
                std_time = np.std(run_times) if len(run_times) > 1 else 0
                avg_value = np.mean(run_values) if run_values else None
                std_value = np.std(run_values) if len(run_values) > 1 else 0
                
                result_entry = {
                    'metric': metric_name,
                    'sample_size': sample_size,
                    'sample_ratio': sample_size / self.total_nodes,
                    'actual_nodes_tested': sample_size,  # Could be less if component extraction
                    'num_successful_runs': len(run_times),
                    'avg_computation_time': avg_time,
                    'std_computation_time': std_time,
                    'avg_metric_value': avg_value,
                    'std_metric_value': std_value,
                    'method': run_methods[0] if run_methods else "unknown",
                    'exceeded_threshold': avg_time > self.max_time_threshold
                }
                
                metric_results.append(result_entry)
                self.results.append(result_entry)
                
                print(f"    📊 Avg time: {avg_time:.2f}±{std_time:.2f}s, Value: {avg_value:.4f}±{std_value:.4f}")
                
                # Stop if consistently exceeding threshold
                if avg_time > self.max_time_threshold:
                    print(f"    🛑 Sample size {sample_size} exceeds threshold, stopping metric testing")
                    break
            else:
                print(f"    ❌ No successful runs for sample size {sample_size}")
        
        return metric_results
    
    def find_optimal_sample_size(self, metric_results: List[Dict], time_budget: float = None) -> Dict:
        """Find the optimal sample size for a metric based on time constraints."""
        if not metric_results:
            return {}
        
        budget = time_budget or self.max_time_threshold
        
        # Filter results within time budget
        viable_results = [r for r in metric_results if r['avg_computation_time'] <= budget]
        
        if not viable_results:
            return {"error": "No sample sizes within time budget"}
        
        # Find largest viable sample size
        optimal = max(viable_results, key=lambda x: x['sample_size'])
        
        return {
            'metric': optimal['metric'],
            'optimal_sample_size': optimal['sample_size'],
            'optimal_sample_ratio': optimal['sample_ratio'],
            'expected_computation_time': optimal['avg_computation_time'],
            'expected_metric_value': optimal['avg_metric_value'],
            'method': optimal['method'],
            'confidence': f"±{optimal['std_metric_value']:.4f}" if optimal['std_metric_value'] else "N/A"
        }
    
    def save_results(self, filename: str = None):
        """Save experimental results to JSON file."""
        if not filename:
            timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
            filename = f"sample_size_experiment_{timestamp}.json"
        
        # Prepare data for JSON serialization
        export_data = {
            'experiment_metadata': {
                'total_nodes': self.total_nodes,
                'total_edges': self.total_edges,
                'max_time_threshold': self.max_time_threshold,
                'experiment_date': datetime.now().isoformat(),
                'graph_type': 'directed' if self.graph.is_directed() else 'undirected'
            },
            'results': self.results
        }
        
        # Save to file
        output_path = Path("../community_detection/output") / filename
        output_path.parent.mkdir(exist_ok=True)
        
        with open(output_path, 'w') as f:
            json.dump(export_data, f, indent=2, default=str)
        
        print(f"Results saved to: {output_path}")
        return output_path

In [7]:
# Initialize experiment
experiment = SampleSizeExperiment(G_main, max_time_threshold=600.0)  # 10 minutes max per test

Experiment Setup:
- Graph: 94,675 nodes, 4,900,260 edges
- Max time threshold: 600.0 seconds


## Initial Testing

In [8]:
# Define sample sizes to test (progressive scaling)
sample_sizes = experiment.generate_sample_sizes(
    start_samples=100, 
    max_samples=8000,  # Conservative max for initial testing
    step_multiplier=1.8
)

print("Sample sizes to test:")
for size in sample_sizes:
    ratio = size / experiment.total_nodes
    print(f"  {size:,} nodes ({ratio:.4f} of total)")

print(f"\nTotal tests planned: {len(sample_sizes)} sample sizes × 4 metrics × 3 runs = {len(sample_sizes) * 4 * 3} individual runs")

Sample sizes to test:
  100 nodes (0.0011 of total)
  180 nodes (0.0019 of total)
  324 nodes (0.0034 of total)
  583 nodes (0.0062 of total)
  1,049 nodes (0.0111 of total)
  1,889 nodes (0.0200 of total)
  3,401 nodes (0.0359 of total)
  6,122 nodes (0.0647 of total)

Total tests planned: 8 sample sizes × 4 metrics × 3 runs = 96 individual runs


In [9]:
# 1. Test DIAMETER metric (2-sweep approximation)
print("🚀 Starting DIAMETER experiments...")
diameter_results = experiment.test_metric_scalability(
    metric_name="diameter",
    metric_func=nx.diameter,
    sample_sizes=sample_sizes,
    num_runs=3
)

🚀 Starting DIAMETER experiments...

Testing DIAMETER scalability

Testing sample size: 100 (0.0011 of total)
  Run 1/3... ✅ 0.00s
  Run 2/3... ✅ 0.00s
  Run 3/3... ✅ 0.00s
    📊 Avg time: 0.00±0.00s, Value: 0.0000±0.0000

Testing sample size: 180 (0.0019 of total)
  Run 1/3... ✅ 0.00s
  Run 2/3... ✅ 0.00s
  Run 3/3... ✅ 0.00s
    📊 Avg time: 0.00±0.00s, Value: 1.3333±0.4714

Testing sample size: 324 (0.0034 of total)
  Run 1/3... ✅ 0.00s
  Run 2/3... ✅ 0.00s
  Run 3/3... ✅ 0.00s
    📊 Avg time: 0.00±0.00s, Value: 2.0000±0.0000

Testing sample size: 583 (0.0062 of total)
  Run 1/3... ✅ 0.00s
  Run 2/3... ✅ 0.00s
  Run 3/3... ✅ 0.00s
    📊 Avg time: 0.00±0.00s, Value: 2.0000±0.0000

Testing sample size: 1,049 (0.0111 of total)
  Run 1/3... ✅ 0.00s
  Run 2/3... ✅ 0.00s
  Run 3/3... ✅ 0.00s
    📊 Avg time: 0.00±0.00s, Value: 5.6667±2.0548

Testing sample size: 1,889 (0.0200 of total)
  Run 1/3... ✅ 0.00s
  Run 2/3... ✅ 0.02s
  Run 3/3... ✅ 0.02s
    📊 Avg time: 0.01±0.01s, Value: 11.6667±2

In [10]:
# 2. Test RADIUS metric (sampling approximation)
print("🚀 Starting RADIUS experiments...")
radius_results = experiment.test_metric_scalability(
    metric_name="radius",
    metric_func=nx.radius,
    sample_sizes=sample_sizes,
    num_runs=3
)

🚀 Starting RADIUS experiments...

Testing RADIUS scalability

Testing sample size: 100 (0.0011 of total)
  Run 1/3... ✅ 0.00s
  Run 2/3... ✅ 0.00s
  Run 3/3... ✅ 0.00s
    📊 Avg time: 0.00±0.00s, Value: 1.0000±0.0000

Testing sample size: 180 (0.0019 of total)
  Run 1/3... ✅ 0.00s
  Run 2/3... ✅ 0.00s
  Run 3/3... ✅ 0.00s
    📊 Avg time: 0.00±0.00s, Value: 1.0000±0.0000

Testing sample size: 324 (0.0034 of total)
  Run 1/3... ✅ 0.00s
  Run 2/3... ✅ 0.00s
  Run 3/3... ✅ 0.00s
    📊 Avg time: 0.00±0.00s, Value: 1.0000±0.0000

Testing sample size: 583 (0.0062 of total)
  Run 1/3... ✅ 0.00s
  Run 2/3... ✅ 0.00s
  Run 3/3... ✅ 0.00s
    📊 Avg time: 0.00±0.00s, Value: 1.0000±0.0000

Testing sample size: 1,049 (0.0111 of total)
  Run 1/3... ✅ 0.00s
  Run 2/3... ✅ 0.00s
  Run 3/3... ✅ 0.00s
    📊 Avg time: 0.00±0.00s, Value: 2.3333±1.2472

Testing sample size: 1,889 (0.0200 of total)
  Run 1/3... ✅ 0.01s
  Run 2/3... ✅ 0.01s
  Run 3/3... ✅ 0.01s
    📊 Avg time: 0.01±0.00s, Value: 5.0000±0.8165

In [11]:
# 3. Test AVERAGE SHORTEST PATH LENGTH metric (sampling approximation)
print("🚀 Starting AVERAGE DISTANCE experiments...")
avg_distance_results = experiment.test_metric_scalability(
    metric_name="avg_distance",
    metric_func=nx.average_shortest_path_length,
    sample_sizes=sample_sizes,
    num_runs=3
)

🚀 Starting AVERAGE DISTANCE experiments...

Testing AVG_DISTANCE scalability

Testing sample size: 100 (0.0011 of total)
  Run 1/3... ✅ 0.00s
  Run 2/3... ✅ 0.00s
  Run 3/3... ✅ 0.00s
    📊 Avg time: 0.00±0.00s, Value: 0.6667±0.4714

Testing sample size: 180 (0.0019 of total)
  Run 1/3... ✅ 0.00s
  Run 2/3... ✅ 0.00s
  Run 3/3... ✅ 0.00s
    📊 Avg time: 0.00±0.00s, Value: 1.2222±0.3143

Testing sample size: 324 (0.0034 of total)
  Run 1/3... ✅ 0.00s
  Run 2/3... ✅ 0.00s
  Run 3/3... ✅ 0.00s
    📊 Avg time: 0.00±0.00s, Value: 1.1111±0.1571

Testing sample size: 583 (0.0062 of total)
  Run 1/3... ✅ 0.00s
  Run 2/3... ✅ 0.00s
  Run 3/3... ✅ 0.00s
    📊 Avg time: 0.00±0.00s, Value: 1.7000±0.1414

Testing sample size: 1,049 (0.0111 of total)
  Run 1/3... ✅ 0.00s
  Run 2/3... ✅ 0.00s
  Run 3/3... ✅ 0.00s
    📊 Avg time: 0.00±0.00s, Value: 3.0159±0.9290

Testing sample size: 1,889 (0.0200 of total)
  Run 1/3... ✅ 0.00s
  Run 2/3... ✅ 0.00s
  Run 3/3... ✅ 0.01s
    📊 Avg time: 0.01±0.00s, Valu

In [12]:
# 4. Test GLOBAL EFFICIENCY metric (sampling approximation)
print("🚀 Starting GLOBAL EFFICIENCY experiments...")
efficiency_results = experiment.test_metric_scalability(
    metric_name="global_efficiency",
    metric_func=nx.global_efficiency,
    sample_sizes=sample_sizes,
    num_runs=3
)

🚀 Starting GLOBAL EFFICIENCY experiments...

Testing GLOBAL_EFFICIENCY scalability

Testing sample size: 100 (0.0011 of total)
  Run 1/3... ✅ 0.00s
  Run 2/3... ✅ 0.00s
  Run 3/3... ✅ 0.00s
    📊 Avg time: 0.00±0.00s, Value: 0.3333±0.4714

Testing sample size: 180 (0.0019 of total)
  Run 1/3... ✅ 0.00s
  Run 2/3... ✅ 0.00s
  Run 3/3... ✅ 0.00s
    📊 Avg time: 0.00±0.00s, Value: 0.6667±0.4714

Testing sample size: 324 (0.0034 of total)
  Run 1/3... ✅ 0.00s
  Run 2/3... ✅ 0.00s
  Run 3/3... ✅ 0.00s
    📊 Avg time: 0.00±0.00s, Value: 0.8426±0.1249

Testing sample size: 583 (0.0062 of total)
  Run 1/3... ✅ 0.00s
  Run 2/3... ✅ 0.00s
  Run 3/3... ✅ 0.00s
    📊 Avg time: 0.00±0.00s, Value: 0.7351±0.0725

Testing sample size: 1,049 (0.0111 of total)
  Run 1/3... ✅ 0.00s
  Run 2/3... ✅ 0.00s
  Run 3/3... ✅ 0.00s
    📊 Avg time: 0.00±0.00s, Value: 0.5667±0.1123

Testing sample size: 1,889 (0.0200 of total)
  Run 1/3... ✅ 0.01s
  Run 2/3... ✅ 0.00s
  Run 3/3... ✅ 0.01s
    📊 Avg time: 0.01±0.00s

## Larger Samples

In [13]:
# Since the initial tests were fast, let's test with much larger sample sizes
larger_sample_sizes = [10000, 15000, 20000, 25000, 30000, 40000, 50000, 60000]

print("🔍 Testing larger sample sizes to find performance limits...")
print("Extended sample sizes:")
for size in larger_sample_sizes:
    ratio = size / experiment.total_nodes
    print(f"  {size:,} nodes ({ratio:.4f} of total)")

🔍 Testing larger sample sizes to find performance limits...
Extended sample sizes:
  10,000 nodes (0.1056 of total)
  15,000 nodes (0.1584 of total)
  20,000 nodes (0.2112 of total)
  25,000 nodes (0.2641 of total)
  30,000 nodes (0.3169 of total)
  40,000 nodes (0.4225 of total)
  50,000 nodes (0.5281 of total)
  60,000 nodes (0.6337 of total)


In [14]:
# Test the most challenging metric first - global efficiency with larger sizes
print("\n🚀 Testing GLOBAL EFFICIENCY with larger sample sizes...")
efficiency_large_results = experiment.test_metric_scalability(
    metric_name="global_efficiency_large",
    metric_func=nx.global_efficiency,
    sample_sizes=larger_sample_sizes,
    num_runs=1  # Reduce runs for larger tests
)


🚀 Testing GLOBAL EFFICIENCY with larger sample sizes...

Testing GLOBAL_EFFICIENCY_LARGE scalability

Testing sample size: 10,000 (0.1056 of total)
  Run 1/1... ✅ 35.73s
    📊 Avg time: 35.73±0.00s, Value: 0.3795±0.0000

Testing sample size: 15,000 (0.1584 of total)
  Run 1/1... ✅ 157.40s
    📊 Avg time: 157.40±0.00s, Value: 0.3129±0.0000

Testing sample size: 20,000 (0.2112 of total)
  Run 1/1... ✅ 411.81s
    📊 Avg time: 411.81±0.00s, Value: 0.3706±0.0000

Testing sample size: 25,000 (0.2641 of total)
  Run 1/1... ✅ 847.99s
    ⚠️ Exceeded time threshold (600.0s), stopping here
    📊 Avg time: 847.99±0.00s, Value: 0.3645±0.0000
    🛑 Sample size 25000 exceeds threshold, stopping metric testing


In [15]:
# Test AVERAGE DISTANCE with larger sample sizes
print("🚀 Testing AVERAGE DISTANCE with larger sample sizes...")
avg_distance_large_results = experiment.test_metric_scalability(
    metric_name="avg_distance_large",
    metric_func=nx.average_shortest_path_length,
    sample_sizes=larger_sample_sizes,
    num_runs=2
)

🚀 Testing AVERAGE DISTANCE with larger sample sizes...

Testing AVG_DISTANCE_LARGE scalability

Testing sample size: 10,000 (0.1056 of total)
  Run 1/2... ✅ 29.01s
  Run 2/2... ✅ 35.62s
    📊 Avg time: 32.31±3.30s, Value: 3.4741±0.0037

Testing sample size: 15,000 (0.1584 of total)
  Run 1/2... ✅ 156.05s
  Run 2/2... ✅ 147.94s
    📊 Avg time: 151.99±4.05s, Value: 3.1903±0.2553

Testing sample size: 20,000 (0.2112 of total)
  Run 1/2... ✅ 413.71s
  Run 2/2... ✅ 387.77s
    📊 Avg time: 400.74±12.97s, Value: 3.0002±0.4213

Testing sample size: 25,000 (0.2641 of total)
  Run 1/2... ✅ 847.01s
    ⚠️ Exceeded time threshold (600.0s), stopping here
    📊 Avg time: 847.01±0.00s, Value: 2.8415±0.0000
    🛑 Sample size 25000 exceeds threshold, stopping metric testing


In [16]:
# Test RADIUS with larger sample sizes
print("🚀 Testing RADIUS with larger sample sizes...")
radius_large_results = experiment.test_metric_scalability(
    metric_name="radius_large",
    metric_func=nx.radius,
    sample_sizes=larger_sample_sizes,
    num_runs=2
)

🚀 Testing RADIUS with larger sample sizes...

Testing RADIUS_LARGE scalability

Testing sample size: 10,000 (0.1056 of total)
  Run 1/2... ✅ 33.77s
  Run 2/2... ✅ 36.12s
    📊 Avg time: 34.94±1.18s, Value: 5.0000±0.0000

Testing sample size: 15,000 (0.1584 of total)
  Run 1/2... ✅ 145.93s
  Run 2/2... ✅ 158.86s
    📊 Avg time: 152.39±6.47s, Value: 3.0000±0.0000

Testing sample size: 20,000 (0.2112 of total)
  Run 1/2... ✅ 374.94s
  Run 2/2... ✅ 416.92s
    📊 Avg time: 395.93±20.99s, Value: 4.0000±0.0000

Testing sample size: 25,000 (0.2641 of total)
  Run 1/2... ✅ 894.17s
    ⚠️ Exceeded time threshold (600.0s), stopping here
    📊 Avg time: 894.17±0.00s, Value: 4.0000±0.0000
    🛑 Sample size 25000 exceeds threshold, stopping metric testing


In [17]:
# Test DIAMETER with larger sample sizes
print("🚀 Testing DIAMETER with larger sample sizes...")
diameter_large_results = experiment.test_metric_scalability(
    metric_name="diameter_large",
    metric_func=nx.diameter,
    sample_sizes=larger_sample_sizes,
    num_runs=2
)

🚀 Testing DIAMETER with larger sample sizes...

Testing DIAMETER_LARGE scalability

Testing sample size: 10,000 (0.1056 of total)
  Run 1/2... ✅ 33.52s
  Run 2/2... ✅ 35.79s
    📊 Avg time: 34.65±1.14s, Value: 8.0000±0.0000

Testing sample size: 15,000 (0.1584 of total)
  Run 1/2... ✅ 155.88s
  Run 2/2... ✅ 160.89s
    📊 Avg time: 158.38±2.51s, Value: 8.5000±0.5000

Testing sample size: 20,000 (0.2112 of total)
  Run 1/2... ✅ 411.52s
  Run 2/2... ✅ 421.95s
    📊 Avg time: 416.73±5.21s, Value: 8.0000±0.0000

Testing sample size: 25,000 (0.2641 of total)
  Run 1/2... ✅ 850.99s
    ⚠️ Exceeded time threshold (600.0s), stopping here
    📊 Avg time: 850.99±0.00s, Value: 6.0000±0.0000
    🛑 Sample size 25000 exceeds threshold, stopping metric testing


## Summary

In [ ]:
# Save results and create final recommendations
results_file = experiment.save_results("sample_size_optimization_results.json")

In [ ]:
def analyze_sample_size_optimization(results_data):
    """
    Analyze graph metric computation results to find optimal sample sizes
    for different time budgets and provide clear recommendations.
    """
    
    # Load results
    if isinstance(results_data, str):
        with open(results_data, 'r') as f:
            data = json.load(f)
    else:
        data = results_data
    
    results = data['results']
    metadata = data['experiment_metadata']
    
    print("🏗️  GRAPH STRUCTURE")
    print("="*80)
    print(f"Total nodes: {metadata['total_nodes']:,}")
    print(f"Total edges: {metadata['total_edges']:,}")
    print(f"Graph type: {metadata['graph_type']}")
    print(f"Max time threshold: {metadata['max_time_threshold']}s")
    print(f"Experiment date: {metadata['experiment_date']}")
    
    # Group results by metric
    metrics = defaultdict(list)
    for result in results:
        base_metric = result['metric'].replace('_large', '')
        metrics[base_metric].append(result)
    
    # Sort each metric's results by sample size
    for metric in metrics:
        metrics[metric].sort(key=lambda x: x['sample_size'])
    
    print("\n\n📊 SCALABILITY ANALYSIS BY METRIC")
    print("="*80)
    
    # Time budgets to analyze (seconds)
    time_budgets = [1, 5, 10, 30, 60, 120, 300, 600]
    optimal_configs = {}
    
    for metric_name, metric_results in metrics.items():
        print(f"\n🎯 {metric_name.upper().replace('_', ' ')}")
        print("-" * 60)
        
        # Display results table
        print("   Sample Size │   Time (s) │   Ratio │  Success │    Value    │ Std Dev")
        print("───────────────┼────────────┼─────────┼──────────┼─────────────┼────────")
        
        for result in metric_results:
            size_str = f"{result['sample_size']:,}"
            time_str = f"{result['avg_computation_time']:.3f}"
            ratio_str = f"{result['sample_ratio']:.4f}"
            success_str = f"{result['num_successful_runs']}/3" if result['num_successful_runs'] < 3 else "✓"
            value_str = f"{result['avg_metric_value']:.4f}"
            std_str = f"{result['std_metric_value']:.4f}"
            
            # Color coding based on time
            if result['avg_computation_time'] > 60:
                time_indicator = "🔴"
            elif result['avg_computation_time'] > 10:
                time_indicator = "🟡"
            else:
                time_indicator = "🟢"
            
            print(f"{size_str:>13} │ {time_str:>9} {time_indicator} │ {ratio_str:>6} │ {success_str:>7} │ {value_str:>10} │ {std_str:>6}")
        
        # Find optimal configurations for different time budgets
        optimal_configs[metric_name] = {}
        
        print(f"\n⏱️  Optimal sample sizes by time budget:")
        for budget in time_budgets:
            # Find largest viable sample within budget
            viable = [r for r in metric_results 
                     if r['avg_computation_time'] <= budget and r['num_successful_runs'] >= 2]
            
            if viable:
                optimal = max(viable, key=lambda x: x['sample_size'])
                optimal_configs[metric_name][budget] = optimal
                efficiency = optimal['sample_size'] / optimal['avg_computation_time']
                print(f"   {budget:>3.0f}s → {optimal['sample_size']:>7,} nodes "
                      f"({optimal['sample_ratio']:.3f}, {efficiency:,.0f} nodes/sec)")
            else:
                print(f"   {budget:>3.0f}s → No viable option")
    
    # Generate recommendations
    print(f"\n\n🏆 RECOMMENDATIONS")
    print("="*80)
    
    # Different use cases
    use_cases = {
        "Quick Analysis": {"budget": 5, "description": "Fast exploratory analysis"},
        "Standard Analysis": {"budget": 30, "description": "Balanced speed vs accuracy"},
        "Detailed Analysis": {"budget": 120, "description": "High accuracy analysis"},
        "Research Quality": {"budget": 600, "description": "Publication-ready results"}
    }
    
    for case_name, case_info in use_cases.items():
        budget = case_info["budget"]
        print(f"\n📋 {case_name} ({case_info['description']})")
        print(f"Time Budget: {budget}s")
        print("-" * 50)
        print("Metric              │ Sample Size │  Ratio │   Time │ Coverage")
        print("────────────────────┼─────────────┼────────┼────────┼─────────")
        
        for metric_name in ['diameter', 'radius', 'avg_distance', 'global_efficiency']:
            if metric_name in optimal_configs and budget in optimal_configs[metric_name]:
                config = optimal_configs[metric_name][budget]
                coverage = config['sample_ratio'] * 100
                print(f"{metric_name.replace('_', ' '):>18} │ {config['sample_size']:>10,} │ "
                      f"{config['sample_ratio']:.4f} │ {config['avg_computation_time']:>6.1f}s │ "
                      f"{coverage:>6.1f}%")
            else:
                print(f"{metric_name.replace('_', ' '):>18} │ {'Not viable':>10} │ {'---':>6} │ {'---':>6} │ {'---':>6}")
    
    # Key insights
    print(f"\n\n💡 KEY INSIGHTS")
    print("="*80)
    
    # Compute scalability insights
    for metric_name, metric_results in metrics.items():
        if len(metric_results) < 2:
            continue
            
        # Find the scaling pattern
        small_samples = [r for r in metric_results if r['sample_size'] < 1000]
        large_samples = [r for r in metric_results if r['sample_size'] > 5000]
        
        if small_samples and large_samples:
            small_time = min(r['avg_computation_time'] for r in small_samples)
            large_time = max(r['avg_computation_time'] for r in large_samples)
            small_size = min(r['sample_size'] for r in small_samples)
            large_size = max(r['sample_size'] for r in large_samples)
            
            time_ratio = large_time / small_time
            size_ratio = large_size / small_size
            
            # Estimate computational complexity
            if size_ratio > 1:
                complexity_exp = round(math.log(time_ratio) / math.log(size_ratio), 1)
                complexity_desc = f"O(n^{complexity_exp})" if complexity_exp > 1 else "Sub-quadratic"
            else:
                complexity_desc = "Cannot determine"
            
            print(f"\n🔍 {metric_name.upper()}:")
            print(f"   • Scales from {small_size:,} to {large_size:,} nodes")
            print(f"   • Time increases {time_ratio:.1f}x for {size_ratio:.1f}x more nodes")
            print(f"   • Estimated complexity: {complexity_desc}")
            
            # Sweet spot recommendation
            sweet_spot = None
            for r in metric_results:
                if 5 < r['avg_computation_time'] < 60:  # 5s to 1min sweet spot
                    if sweet_spot is None or r['sample_size'] > sweet_spot['sample_size']:
                        sweet_spot = r
            
            if sweet_spot:
                coverage = sweet_spot['sample_ratio'] * 100
                print(f"   • Recommended sample: {sweet_spot['sample_size']:,} nodes "
                      f"({coverage:.1f}% coverage, ~{sweet_spot['avg_computation_time']:.1f}s)")
    
    print(f"\n\n⚡ PERFORMANCE SUMMARY")
    print("="*80)
    print("• Diameter and radius scale poorly - use sampling for large graphs")
    print("• Average distance is more tractable but still requires careful sampling")  
    print("• Global efficiency computation time varies significantly with graph structure")
    print("• For graphs with 94K+ nodes, sample sizes of 1K-10K provide good balance")
    print("• Consider approximate algorithms for very large graphs or tight time budgets")
    
    return optimal_configs

In [44]:
# Load JSON file
with open("../community_detection/output/sample_size_optimization_results.json", "r") as f:
    sample_data = json.load(f)

# Pass data to your analysis function
result = analyze_sample_size_optimization(sample_data)

🏗️  GRAPH STRUCTURE
Total nodes: 94,675
Total edges: 4,900,260
Graph type: directed
Max time threshold: 600.0s
Experiment date: 2025-09-23T18:49:03.639972


📊 SCALABILITY ANALYSIS BY METRIC

🎯 DIAMETER
------------------------------------------------------------
   Sample Size │   Time (s) │   Ratio │  Success │    Value    │ Std Dev
───────────────┼────────────┼─────────┼──────────┼─────────────┼────────
          100 │     0.000 🟢 │ 0.0011 │       ✓ │     0.0000 │ 0.0000
          180 │     0.000 🟢 │ 0.0019 │       ✓ │     1.3333 │ 0.4714
          324 │     0.000 🟢 │ 0.0034 │       ✓ │     2.0000 │ 0.0000
          583 │     0.000 🟢 │ 0.0062 │       ✓ │     2.0000 │ 0.0000
        1,049 │     0.000 🟢 │ 0.0111 │       ✓ │     5.6667 │ 2.0548
        1,889 │     0.012 🟢 │ 0.0200 │       ✓ │    11.6667 │ 2.4944
        3,401 │     0.242 🟢 │ 0.0359 │       ✓ │     9.6667 │ 0.4714
        6,122 │     4.931 🟢 │ 0.0647 │       ✓ │    10.0000 │ 0.8165
       10,000 │    34.653 🟡 │ 0.1056 │ 